# Daily Pizza Sales Prediction


#### Project Workflow
1. Understand the Dataset
- Review all columns and their meanings (you’ve already done this — great start!)
- Identify which variables are:
- Independent (features): weather, promotions, school status, holidays, etc.
- Dependent (target): daily_sales
2. Clean and Prepare the Data
- Check for missing values or anomalies (e.g., nulls in temperature or sales)
- Convert date column to datetime format
- Create new features if needed:
- Week number
- Is exam week
- Ramadan or Lent flag (already modeled, but you can double-check)
3. Explore the Data (EDA)
Use visualizations to uncover patterns:
- 📈 Line plots of sales over time
- 📊 Bar charts comparing average sales by:
- Day of week
- Month
- Holiday vs non-holiday
- School in session vs strike
- 📉 Boxplots to see sales distribution by weather or promotion
- 📌 Correlation heatmap to see which features influence sales most
4. Model Sales Drivers
- Use regression models (e.g., Linear Regression, Random Forest, XGBoost) to predict daily_sales
- Evaluate feature importance: which variables drive sales the most?
- Try time series models (e.g., ARIMA, Prophet) if you're forecasting future sales
5. Segment Your Insights
- Compare sales during:
- Strike vs normal periods
- Ramadan vs non-Ramadan
- Exam weeks vs regular weeks
- Identify high-performing days (e.g., Fridays with promotions)
6. Make Recommendations
Based on your findings, suggest:
- Best times to run promotions
- How to prepare for low-traffic periods (e.g., strikes, Lent)
- Staffing or inventory adjustments based on seasonality
7. Present Your Work
- Create a dashboard (Excel, Power BI, or Tableau)
- Summarize key insights in a slide deck or report
- Include visuals, trends, and actionable takeaways


# Data Cleaning

In [ ]:
# Importing Libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
#Importing Dataset
df = pd.read_csv('pizza_sales_2021_2025.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.drop(['ramadan', 'lent'], axis=1, inplace=True)

In [ ]:
print(df.info())

In [ ]:
df.describe()

In [ ]:
print(df.isnull().sum())

`Note: `In the `public_holiday_name` column, missing values likely mean “not a public holiday” — which is perfectly valid. So these aren’t errors or gaps in data collection, they’re just non-holiday days.
- Since `is_holiday` as a Boolean column — so you can use that to filter or group.
- Also, When `is_holiday` = False, it’s expected that public_holiday_name = NaN.





In [ ]:
# Check for Duplicates
print(df.duplicated().sum())

In [ ]:
#Check and FIx Data Types
print(df.dtypes)

In [ ]:
# Convert Date column to datetime dtype
df['date'] = pd.to_datetime(df['date'], errors='coerce')


In [ ]:
#Convert Category columns to category dtype
cat_cols = ['day_of_week', 'month', 'public_holiday_name', 'university_calendar_status', 'weather']
for col in cat_cols:
    df[col] = df[col].astype('category')
    

In [ ]:
#Convert Boolean columns to bool dtype
bool_cols = ['is_weekend', 'is_holiday', 'is_school_in_session', 'promotion']
for col in bool_cols:
    df[col] = df[col].astype('bool')

In [ ]:
#Convert Numeric Columns to floats type
num_cols = ['temperature_C', 'foot_traffic_index', 'student_density_index', 'daily_sales_NGN', 'transactions_count', 'avg_order_value_NGN']
for col in num_cols:  
    df[col] = df[col].astype('float64')


In [ ]:
print(df.dtypes)

In [ ]:
# Checking Time Continuity to ensure no missing dates

# Create a complete date range
full_range = pd.date_range(start=df['date'].min(), end=df['date'].max())

# Compare with actual dates
missing_dates = full_range.difference(df['date'])

print(f"Missing dates: {missing_dates}")


## Exploratory Data Analysis

In [ ]:
# plot a histogram for each numerical attribute
df.hist(bins=50, figsize=(20,15))
plt.show()


In [ ]:
# Summary stats
print(
    df[['daily_sales_NGN', 'temperature_C', 'student_density_index', 'foot_traffic_index', 'transactions_count', 'avg_order_value_NGN']].describe()
)


# Visualizing boxplots for numerical columns
for col in ['daily_sales_NGN', 'temperature_C', 'student_density_index', 'foot_traffic_index', 'transactions_count', 'avg_order_value_NGN']:
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot of {col}')
    plt.show()

### Looking for Correlations

In [ ]:
corr_matrix = df[num_cols].corr()

In [ ]:
# Visualizing the correlation matrix using a heatmap

plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numeric Features')
plt.show()

Understand the `correlation martix` before continuing with Data Preprocessing

## Data Preprocessing

- Handle Missing Values
- Define Variables
- Feature Engineering & Featur Scaling where necessary
- Encoding Categorical data


`Note:` For Feature engineering on this project;

 Addressing Special Non-Public HolidaysYou should create a new binary feature specifically to capture the effect of fixed, non-official holidays that dramatically influence consumer spending and dining habits.
 1. Create a New Feature: Festive_Day_FlagInstead of trying to fit Valentine's Day into the Is_Holiday column (which should be reserved only for nationally recognized public holidays), you should create a separate binary flag:
 
- New Column Name:  Festive_Day_FlagBinary
- DataType: (0 or 1)
- Description : 1 if the date is a major, fixed festive day known to influence dining, 0 otherwise
- Dates to Flag (Examples): February 14th (Valentine's), Mother's Day, Father's Day, New Year's Eve (Dec 31st).


`Note:` for Encoding Categorical Data;
1. One-Hot Encoding (Dummy Variables)
Best for tree-based models (Random Forest, XGBoost) and linear models.
`df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)`

- drop_first=True avoids multicollinearity by removing one category per feature.
- This turns each category into a binary column (0 or 1).


In [ ]:
print(df.columns)


In [ ]:
#Feature Engineering #Lag Features
# Sales lags
df['sales_lag_1'] = df['daily_sales_NGN'].shift(1)
df['sales_lag_7'] = df['daily_sales_NGN'].shift(7)
df['sales_lag_30'] = df['daily_sales_NGN'].shift(30)

# Foot traffic lags
df['traffic_lag_1'] = df['foot_traffic_index'].shift(1)
df['traffic_lag_7'] = df['foot_traffic_index'].shift(7)

# Transactions lags
df['transactions_lag_1'] = df['transactions_count'].shift(1)
df['transactions_lag_7'] = df['transactions_count'].shift(7)


In [ ]:
#Rolling averages; helps the model understand recent trends and smooth out daily noise.

# Rolling averages for sales
df['sales_7d_avg'] = df['daily_sales_NGN'].rolling(window=7).mean()
df['sales_30d_avg'] = df['daily_sales_NGN'].rolling(window=30).mean()

# Rolling averages for foot traffic and transactions
df['traffic_7d_avg'] = df['foot_traffic_index'].rolling(window=7).mean()
df['transactions_7d_avg'] = df['transactions_count'].rolling(window=7).mean()
df.head(15)

Find answeres to this later;  If I drop the rows of the columns with NaN values, how will the model learn from the detials of their other colums with useful detials

In [ ]:
# Adding non official public holidays or festive days that migh affect daily sales

festive_days = [
    '2021-02-14', '2021-03-14', '2021-06-20', '2021-12-24', '2021-12-31',
    '2022-02-14', '2022-03-27', '2022-06-19', '2022-12-24', '2022-12-31',
    '2023-02-14', '2023-03-19', '2023-06-18', '2023-12-24', '2023-12-31',
    '2024-02-14', '2024-03-10', '2024-06-16', '2024-12-24', '2024-12-31',
    '2025-02-14', '2025-03-30', '2025-06-15', '2025-12-24', '2025-12-31'
]

In [ ]:
# Adding the names of the non official public holidays

festive_names = {
    '2021-02-14': "Valentine's Day",
    '2021-03-14': "Mother's Day",
    '2021-06-20': "Father's Day",
    '2021-12-24': "Christmas Eve",
    '2021-12-31': "New Year's Eve",
    '2022-02-14': "Valentine's Day",
    '2022-03-27': "Mother's Day",
    '2022-06-19': "Father's Day",
    '2022-12-24': "Christmas Eve",
    '2022-12-31': "New Year's Eve",
    '2023-02-14': "Valentine's Day",
    '2023-03-19': "Mother's Day",
    '2023-06-18': "Father's Day",
    '2023-12-24': "Christmas Eve",
    '2023-12-31': "New Year's Eve",
    '2024-02-14': "Valentine's Day",
    '2024-03-10': "Mother's Day",
    '2024-06-16': "Father's Day",
    '2024-12-24': "Christmas Eve",
    '2024-12-31': "New Year's Eve",
    '2025-02-14': "Valentine's Day",
    '2025-03-30': "Mother's Day",
    '2025-06-15': "Father's Day",
    '2025-12-24': "Christmas Eve",
    '2025-12-31': "New Year's Eve"
}

In [ ]:
#Convert Date Column to String Format for mapping
df['date_str'] = df['date'].dt.strftime('%Y-%m-%d') 


In [ ]:
# Update the colums to include festive days and their names
df.loc[df['date_str'].isin(festive_days), 'is_holiday'] = True


# Append festive names to public holiday name
df['public_holiday_name'] = df.apply(
    lambda row: festive_names[row['date_str']] if row['date_str'] in festive_names
    else row['public_holiday_name'], axis=1
)

In [ ]:
df.drop(columns=['date_str'], inplace=True) #drop the date column in str format

In [ ]:
df

- Cyclical Time Encoding — turn day_of_week and month into sine/cosine features
- Train-Test Split — prepare your data for modeling
- Feature Selection — identify the most predictive features
- Modeling — build and evaluate your forecasting model


In [ ]:
# Handling Missing Values after Feature Engineering
print(df.isnull().sum())

In [ ]:
# Handling NaNs in 'public_holiday_name'
df['public_holiday_name'] = df['public_holiday_name'].fillna('None')


In [ ]:

df = df.dropna().copy()  # drop rows with any remaining NaNs

In [ ]:
print(df.isnull().sum())

### Encoding Categorical Data

In [ ]:
# cyclical encoding of date features
df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6
df['month'] = df['date'].dt.month            # January=1, December=12


In [ ]:
# Apply sine and cosine transformations
import numpy as np

# Day of week (7-day cycle)
df['dow_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['dow_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

# Month (12-month cycle)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)


In [ ]:
df.head()

In [ ]:
df['university_calendar_status'].unique()


In [ ]:
print(df.dtypes)


In [ ]:
# Encoding categorical columns 
df = pd.get_dummies(df, columns=[df.columns[6], df.columns[8]], drop_first=True)

# Encoding boolean columns as integers
bool_cols = ['is_weekend', 'is_holiday', 'is_school_in_session', 'promotion']
for col in bool_cols:
    df[col] = df[col].astype(int)


In [ ]:
print(df.dtypes)

In [ ]:
df.head()

### Interaction Features

In [ ]:
print(df.dtypes)

In [ ]:
df['traffic_promo'] = df['foot_traffic_index'] * df['promotion']
df['holiday_promo'] = df['is_holiday'] * df['promotion']
df['weekend_promo'] = df['is_weekend'] * df['promotion']
df['school_promo'] = df['is_school_in_session'] * df['promotion']
df['holiday_traffic'] = df['is_holiday'] * df['foot_traffic_index']


In [ ]:
df.head()

In [ ]:
#Separating Features and Target Variable

X = df.drop(columns=['daily_sales_NGN', 'date', 'public_holiday_name']) #Features
y = df['daily_sales_NGN'] #Target


## Train/Test Split

In [ ]:
# Splitting the dataset into Training set and Test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [ ]:
# Feature Scaling for Linear Regression Model

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Train or Fit the Model into the Training Dataset;
- Linear Regression Model
- Random Forest 
- XGboost

### Linear Regression Model

In [ ]:
# Fit the Linear Regression Model on the Scaled Data

from sklearn.linear_model import LinearRegression  
regressor = LinearRegression()
regressor.fit(X_train_scaled, y_train)


In [ ]:
# Predicting the Test Set Results
y_pred = regressor.predict(X_test_scaled)

y_pred

In [ ]:
y_test.head(10)

In [ ]:
# Visualizing the Linear Regression Predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label='Actual', linewidth=2)
plt.plot(y_pred, label='Predicted', linestyle='--')
plt.title('Linear Regression: Actual vs Predicted Sales')
plt.xlabel('Time')
plt.ylabel('Sales (NGN)')
plt.legend()
plt.show()

In [ ]:
# Evaluating model performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

### Random Forest Regression

In [ ]:
# Fitting the Random Forest Regression Model into the Dataset
from sklearn.ensemble import RandomForestRegressor
rf_regressor = RandomForestRegressor(n_estimators=300, random_state=0)
rf_regressor.fit(X_train, y_train)


In [ ]:
rf_y_pred = rf_regressor.predict(X_test)


In [ ]:
rf_y_pred

In [ ]:
y_test

In [ ]:
# Visualizing the Random Forest Predictions on Test Set
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label='Actual', linewidth=2)
plt.plot(rf_y_pred, label='Predicted(RF)', linestyle='--')
plt.title('Random Forest Regression: Actual vs Predicted Sales')
plt.xlabel('Time')
plt.ylabel('Sales (NGN)')
plt.legend()
plt.show()



 # Visualizing the Random Forest Predictions on Train Set
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(y_train.values, label='Actual', linewidth=2)
plt.plot(rf_regressor.predict(X_train), label='Predicted (RF)', linestyle='--')
plt.title('Random Forest: Actual vs Predicted Sales')
plt.xlabel('Time')
plt.ylabel('Sales (NGN)')
plt.legend()
plt.show()

In [ ]:
# Evaluting model performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae_rf = mean_absolute_error(y_test, rf_y_pred)
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_y_pred))
r2_rf = r2_score(y_test, rf_y_pred)

print(f"Random Forest MAE: {mae_rf:.2f}")
print(f"Random Forest RMSE: {rmse_rf:.2f}")
print(f"Random Forest R²: {r2_rf:.2f}")


In [ ]:
# Feature Importance from Random Forest Model
import pandas as pd

feature_importance = rf_regressor.feature_importances_
features = X_train.columns

importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

In [ ]:
# Visualizing Feature Importance
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.gca().invert_yaxis()
plt.title('Random Forest Feature Importance')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

In [ ]:
# Model Refinement; Filter out the features with low importance and keep the ones with high importance(top 10)

top_features = importance_df['Feature'].head(10).tolist()
X_train_reduced = X_train[top_features]
X_test_reduced = X_test[top_features]



In [ ]:
print(top_features)

In [ ]:
# Try retraining with only the top 10 features

rf_regressor_reduced = RandomForestRegressor(n_estimators=300, random_state=0)
rf_regressor_reduced.fit(X_train_reduced, y_train)

In [ ]:
rf_y_pred_reduced = rf_regressor_reduced.predict(X_test_reduced)


In [ ]:
rf_y_pred_reduced

In [ ]:
# Evaluting the Refined model performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae_rf = mean_absolute_error(y_test, rf_y_pred_reduced)
rmse_rf = np.sqrt(mean_squared_error(y_test, rf_y_pred_reduced))
r2_rf = r2_score(y_test, rf_y_pred_reduced)

print(f"Random Forest MAE: {mae_rf:.2f}")
print(f"Random Forest RMSE: {rmse_rf:.2f}")
print(f"Random Forest R²: {r2_rf:.2f}")


### XGBoost

In [ ]:
# Fitting the XGBoost Regression Model into the Dataset using the redudced feature set
from xgboost import XGBRegressor

xgb_regressor = XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=6, random_state=0)
xgb_regressor.fit(X_train_reduced, y_train)

In [ ]:
xgb_y_pred = xgb_regressor.predict(X_test_reduced)

In [ ]:
xgb_y_pred

In [ ]:
# Visualizing the XGBoost Predictions on Test Set
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label='Actual', linewidth=2)
plt.plot(xgb_y_pred, label='Predicted(RF)', linestyle='--')
plt.title('XGBoost: Actual vs Predicted Sales')
plt.xlabel('Time')
plt.ylabel('Sales (NGN)')
plt.legend()
plt.show()



 # Visualizing the XGBoost Predictions on Train Set
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(y_train.values, label='Actual', linewidth=2)
plt.plot(xgb_regressor.predict(X_train_reduced), label='Predicted (RF)', linestyle='--')
plt.title('XGBoost: Actual vs Predicted Sales')
plt.xlabel('Time')
plt.ylabel('Sales (NGN)')
plt.legend()
plt.show()

In [ ]:
# Evaluting XGBoost model performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae_xgb = mean_absolute_error(y_test, xgb_y_pred)
rmse_xgb = np.sqrt(mean_squared_error(y_test, xgb_y_pred))
r2_xgb = r2_score(y_test, xgb_y_pred)

print(f"XGBoost MAE: {mae_xgb:.2f}")
print(f"XGBoost RMSE: {rmse_xgb:.2f}")
print(f"XGBoost R²: {r2_xgb:.2f}")

In [ ]:
# Fine Tiuning and Hyperparameter Optimization can be done using GridSearchCV or RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

xgb = XGBRegressor()
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_reduced, y_train)

best_model = grid_search.best_estimator_

In [ ]:
# Evaluate the best model
y_pred_best = best_model.predict(X_test_reduced)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae_best = mean_absolute_error(y_test, y_pred_best)
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
r2_best = r2_score(y_test, y_pred_best)

print(f"Best XGBoost MAE: {mae_best:.2f}")
print(f"Best XGBoost RMSE: {rmse_best:.2f}")
print(f"Best XGBoost R²: {r2_best:.2f}")

## Finalize Model Pipeline


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

class CustomPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, top_features):
        self.top_features = top_features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()

        # --- Feature Engineering ---
        df['sales_lag_1'] = df['sales'].shift(1)
        df['sales_lag_7'] = df['sales'].shift(7)
        df['sales_lag_30'] = df['sales'].shift(30)
        df['traffic_lag_1'] = df['traffic'].shift(1)
        df['traffic_lag_7'] = df['traffic'].shift(7)
        df['transactions_lag_1'] = df['transactions'].shift(1)
        df['transactions_lag_7'] = df['transactions'].shift(7)

        df['sales_7d_avg'] = df['sales'].rolling(7).mean()
        df['sales_30d_avg'] = df['sales'].rolling(30).mean()
        df['traffic_7d_avg'] = df['traffic'].rolling(7).mean()
        df['transactions_7d_avg'] = df['transactions'].rolling(7).mean()

        # --- Cyclical Time Encoding ---
        df['date'] = pd.to_datetime(df['date'])
        df['day_of_week'] = df['date'].dt.dayofweek
        df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

        # --- One-Hot Encoding ---
        df = pd.get_dummies(df, columns=['store_type', 'region'], drop_first=True)

        # --- Drop rows with NaNs from lag/rolling ---
        df = df.dropna()

        # --- Select Top Features ---
        return df[self.top_features]

In [ ]:
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import joblib

# Replace with your actual top 10 feature names
top_10_features = ['sales_lag_1', 'sales_lag_7', 'traffic_lag_7', 'sales_7d_avg', 'sales_30d_avg', 'transactions_lag_7', 'traffic_7d_avg'. 'transactions_7d_avg', 'day_sin', 'store_type_TypeB']  # example

pipeline = Pipeline(steps=[
    ('preprocessing', CustomPreprocessor(top_features=top_10_features)),
    ('model', best_model)  # your tuned XGBoost model
])

# Fit on raw training data
pipeline.fit(raw_X_train, y_train)

# Save the full pipeline
joblib.dump(pipeline, 'xgboost_full_pipeline_raw.pkl')

In [ ]:
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import joblib

# Build pipeline with only the model
pipeline = Pipeline(steps=[
    ('model', best_model)
])

# Train the pipeline on your reduced, preprocessed data
pipeline.fit(X_train_reduced, y_train)

# Save the pipeline
joblib.dump(pipeline, 'xgboost_final_pipeline.pkl')

In [ ]:
## I still want to make some changeseg add all the pre preocessing steps to the model pipeline before deploying it.

## Launch the Model on Streamlit app

 Let's Launch the model..
  